# Dataset prepration for Emory NLP

In [13]:
import os
import glob
import json
import random
from pathlib import Path
import pandas as pd # type: ignore

### Read data files

In [5]:
df_train = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp/data_files/emorynlp_train_final.csv")
df_test = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp/data_files/emorynlp_test_final.csv")
df_dev = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp/data_files/emorynlp_dev_final.csv")

In [9]:
df_dev

,Utterance,Speaker,Emotion,Scene_ID,Utterance_ID,Season,Episode,Start_Time,End_Time
0,Coffee.,['Rachel Green'],Neutral,1,1,1,15,00:00:03.795,00:00:05.004
1,Thank you.,['Joey Tribbiani'],Neutral,1,2,1,15,00:00:05.171,00:00:07.423
2,Cappuccino.,['Rachel Green'],Neutral,1,3,1,15,00:00:07.590,00:00:08.757
3,Grazie.,['Ross Geller'],Neutral,1,4,1,15,00:00:08.925,00:00:11.677
4,And a nice hot cider for Monica.,['Rachel Green'],Neutral,1,5,1,15,00:01:27.253,00:01:33.383
...,...,...,...,...,...,...,...,...,...
949,It's better! You can't go to a museum in your ...,['Joey Tribbiani'],Joyful,29,3,4,21,00:21:37.379,00:21:44.135
950,"Well, You could, but... probably just the one ...",['Chandler Bing'],Joyful,29,4,4,21,00:21:45.220,00:21:47.555
951,"I bet we could get videos of all the sites, ge...",['Joey Tribbiani'],Joyful,29,5,4,21,00:21:49.641,00:21:55.604
952,"If we do that, we gotta get Die Hard.",['Chandler Bing'],Joyful,29,6,4,21,00:21:56.189,00:21:58.190


In [12]:
df_dev.Emotion.value_counts()

Emotion
Neutral     322
Joyful      205
Scared      127
Mad          97
Peaceful     82
Powerful     70
Sad          51
Name: count, dtype: int64

### Paths

In [20]:
current_dir = Path.cwd()
dataset_dir = current_dir / "emotion_analysis_comics" / "emory_nlp" / "datasets"

In [21]:
dataset_dir

PosixPath('/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp/datasets')

### Prepare prompts

In [38]:
# Formatting Fx
# Build questoin
# Build answer

In [34]:
def formatting_fct(instruction="", input="", output=""):
    
    prompt_d ={
            
        "instruction": f"""{instruction}""",
        "input": f"""{input}""",
        "output": f"""{output}"""
            
        }
    
    return prompt_d

In [35]:
def build_instruction():

    #results = json.dumps(["emotion_class (str)"] * nr_utterances)

    instruction = f"""### You are an expert in Emotion Analysis. You are given an utterance from a tv show enclosed by <UT></UT> tags. Your task is to classify each utterance as one the following emotion classes: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful" or "Neutral". You must return an emotion class in following JSON format: {{"emotion_class": ["emotion_class (str)"]}} where "emotion_classes (str)" is replaced by one of the following abbreviated emotion class labels: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful" or "Neutral". 
"""    
    return instruction

In [36]:
def build_tagged_text(utterance):

    # tagged_utterances_l = []

    # for idx, utterance in enumerate(utterances_l):
        
    #     start_tag = "<UT" + str(idx+1) + ">"
    #     end_tag = "</UT" + str(idx+1) + ">"
    #     tagged_utterance = start_tag + utterance + end_tag
    #     tagged_utterances_l.append(tagged_utterance)
        
    # tagged_title_text = ''.join(tagged_utterances_l)
    
    tagged_utterance = "<UT>" + utterance + "</UT>"
    question = f"""### Here is the utterance from a tv show: {tagged_utterance}"""

    return question

In [37]:
# utterance_emotions_l = []
#     emotion_class_labels = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy"]

#     if utterance_emotions == 'Neutral':
        
#         utterance_emotions_l.append([utterance_emotions])
    
#     else:
#         utterance_emotions = utterance_emotions.split("-")
       
#         #emotion_annotation_l = []

#         for idx, emotion_annotation in enumerate(utterance_emotions):

#             if '0' not in emotion_annotation:
         
#                 #emotion_annotation_l.append(emotion_class_labels[idx])
#                 utterance_emotions_l.append(emotion_annotation[:-1])
            
#         #title_emotions_l.append(emotion_annotation_l)

In [38]:
def build_answer(utterance_emotion):    
                

    return json.dumps({"emotion_class": [utterance_emotion]})

### Build Data Files

In [39]:
#df_train = df[df.split == 'TRAIN'].reset_index()

data_file_train = []

for index, _ in df_train.iterrows():
    
    i = index

    instruction = build_instruction()
    question = build_tagged_text(df_train.iloc[i].Utterance)
    answer = build_answer(df_train.iloc[i].Emotion)
    
    data_file_train.append( formatting_fct(instruction, question, answer) )

In [40]:
len(data_file_train)

7551

In [41]:
for i in range(5):
    print(data_file_train[i])

{'instruction': '### You are an expert in Emotion Analysis. You are given an utterance from a tv show enclosed by <UT></UT> tags. Your task is to classify each utterance as one the following emotion classes: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful" or "Neutral". You must return an emotion class in following JSON format: {"emotion_class": ["emotion_class (str)"]} where "emotion_classes (str)" is replaced by one of the following abbreviated emotion class labels: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful" or "Neutral". \n', 'input': "### Here is the utterance from a tv show: <UT>What you guys don't understand is, for us, kissing is as important as any part of it.</UT>", 'output': '{"emotion_class": ["Joyful"]}'}
{'instruction': '### You are an expert in Emotion Analysis. You are given an utterance from a tv show enclosed by <UT></UT> tags. Your task is to classify each utterance as one the following emotion classes: "Mad", "Scared", "Sad", "Powerful", "Peacef

In [42]:
#df_test = df[df.split == 'TEST'].reset_index()

data_file_test = []

for index, _ in df_test.iterrows():
    
    i = index

    instruction = build_instruction()
    question = build_tagged_text(df_test.iloc[i].Utterance)
    answer = build_answer(df_test.iloc[i].Emotion)
    
    data_file_test.append( formatting_fct(instruction, question, answer) )

In [46]:
len(data_file_test)

984

In [44]:
for i in range(5):
    print(data_file_test[i])

{'instruction': '### You are an expert in Emotion Analysis. You are given an utterance from a tv show enclosed by <UT></UT> tags. Your task is to classify each utterance as one the following emotion classes: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful" or "Neutral". You must return an emotion class in following JSON format: {"emotion_class": ["emotion_class (str)"]} where "emotion_classes (str)" is replaced by one of the following abbreviated emotion class labels: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful" or "Neutral". \n', 'input': "### Here is the utterance from a tv show: <UT>I'm supposed to attach a brackety thing to the side things, using a bunch of these little worm guys. I have no brackety thing, I see no whim guys whatsoever and- I cannot feel my legs.</UT>", 'output': '{"emotion_class": ["Mad"]}'}
{'instruction': '### You are an expert in Emotion Analysis. You are given an utterance from a tv show enclosed by <UT></UT> tags. Your task is to classify 

In [45]:
#df_test = df[df.split == 'TEST'].reset_index()

data_file_dev = []

for index, _ in df_dev.iterrows():
    
    i = index

    instruction = build_instruction()
    question = build_tagged_text(df_dev.iloc[i].Utterance)
    answer = build_answer(df_dev.iloc[i].Emotion)
    
    data_file_dev.append( formatting_fct(instruction, question, answer) )

In [47]:
len(data_file_dev)

954

In [48]:
for i in range(5):
    print(data_file_dev[i])

{'instruction': '### You are an expert in Emotion Analysis. You are given an utterance from a tv show enclosed by <UT></UT> tags. Your task is to classify each utterance as one the following emotion classes: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful" or "Neutral". You must return an emotion class in following JSON format: {"emotion_class": ["emotion_class (str)"]} where "emotion_classes (str)" is replaced by one of the following abbreviated emotion class labels: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful" or "Neutral". \n', 'input': '### Here is the utterance from a tv show: <UT>Coffee.</UT>', 'output': '{"emotion_class": ["Neutral"]}'}
{'instruction': '### You are an expert in Emotion Analysis. You are given an utterance from a tv show enclosed by <UT></UT> tags. Your task is to classify each utterance as one the following emotion classes: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful" or "Neutral". You must return an emotion class in following JSO

### Create and save JSON files

In [50]:
dataset_dir.mkdir(parents=True, exist_ok=True)

In [51]:
file_path = Path(dataset_dir) / "emorynlp_utterance_train.json"

with open(file_path, 'w') as file:
    
    json.dump(data_file_train, file)

In [52]:
file_path = Path(dataset_dir) / "emorynlp_utterance_test.json"

with open(file_path, 'w') as file:
    
    json.dump(data_file_test, file)

In [53]:
file_path = Path(dataset_dir) / "emorynlp_utterance_dev.json"

with open(file_path, 'w') as file:
    
    json.dump(data_file_dev, file)